In [1]:


from __future__ import division, absolute_import
import re
import numpy as np
from dataset_loader import DatasetLoader
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected, flatten
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d
from tflearn.layers.merge_ops import merge
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
from constants import *
from os.path import isfile, join
import random
import sys

import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *

def residual_block(f, x, ker_reg=None, bias_reg=None):
    short_cut = x
#             block 1
    x = Conv2D(f , kernel_size=3, strides=2, data_format="channels_last", padding='same',
               kernel_regularizer=ker_reg, bias_regularizer=bias_reg)(x)
    x = BatchNormalization(axis=-1)(x)
    x = Activation('relu')(x)
#             block 2, no activation function
    x = Conv2D(f , kernel_size=3, strides=1, data_format="channels_last", padding='same',
              kernel_regularizer=ker_reg, bias_regularizer=bias_reg)(x)
    x = BatchNormalization(axis=-1)(x)
#             short_cut
    short_cut = Conv2D(f , kernel_size=3, strides=2, data_format="channels_last", padding='same',
                      kernel_regularizer=ker_reg, bias_regularizer=bias_reg)(short_cut)
    short_cut = BatchNormalization(axis=-1)(short_cut)
#             add
    x = add([x, short_cut])
    x = Activation('relu')(x)
    return x
def res_cnn():
    l2_ker_reg = keras.regularizers.l2(0.01)
    l2_bias_reg = keras.regularizers.l2(0.01)
    
    face_input = Input(shape=[SIZE_FACE, SIZE_FACE, 1], name='face_input')
    x = face_input
    for i in range(4):
        short_cut = x
        f = 128 * (2**i)
        x = residual_block(f, x, l2_ker_reg, l2_bias_reg)
#     global pooling
    x = AveragePooling2D(3)(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
#     x = AlphaDropout(0.3)(x)
#     x = Dense(256, activation='relu')(x)
    emo_class = Dense(len(EMOTIONS), activation='softmax')(x)

    model = Model(inputs=face_input, outputs=emo_class)
    
    return model

def std_cnn():
    ker_reg = l2_ker_reg = keras.regularizers.l2(0.01)
    bias_reg = l2_bias_reg = keras.regularizers.l2(0.01)
    
    face_input = Input(shape=[SIZE_FACE, SIZE_FACE, 1], name='face_input')
    x = face_input
    for i in range(4):
        f = 128 * (2**i)
        
        x = Conv2D(f , kernel_size=3, strides=2, data_format="channels_last", padding='same',
              kernel_regularizer=ker_reg, bias_regularizer=bias_reg)(x)
    
        x = BatchNormalization(axis=-1)(x)
        x = Activation('relu')(x)
    x = Flatten()(x)
    x = AlphaDropout(0.3)(x)
    x = Dense(1024, activation='relu')(x)
    emo_class = Dense(len(EMOTIONS), activation='softmax')(x)

    model = Model(inputs=face_input, outputs=emo_class)
    return model

class EmotionRecognition:

    def __init__(self):
        self.dataset = DatasetLoader()

    def build_network(self):
        # Smaller 'AlexNet'
        # https://github.com/tflearn/tflearn/blob/master/examples/images/alexnet.py
        print('[+] Building CNN')
        
#         self.model = res_cnn()
        
        self.model = std_cnn()
        
        opt = Adam(lr=0.001)
        self.model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        
        self.model.summary()
#         self.load_model()
    
    def load_saved_dataset(self):
        self.dataset.load_from_save()
        print('[+] Dataset found and loaded')

    def start_training(self):
        self.load_saved_dataset()
        self.build_network()
        if self.dataset is None:
            self.load_saved_dataset()
        # Training
        print('[+] Training network')
        cks = keras.callbacks.ModelCheckpoint('./data/checkpoint', monitor='val_loss', save_best_only=True )
        logs = keras.callbacks.TensorBoard(log_dir='./logs/res/')
        self.model.fit(
            self.dataset.images, self.dataset.labels,
            validation_split = 0.1,
#             validation_data=(self.dataset.images_test,
#                             self.dataset.labels_test),
            callbacks=[cks, logs],
            epochs=100,
            batch_size=256
        )

    def predict(self, image):
        if image is None:
            return None
        image = image.reshape([-1, SIZE_FACE, SIZE_FACE, 1])
        return self.model.predict(image)

    def save_model(self):
        self.model.save(join(SAVE_DIRECTORY, SAVE_MODEL_FILENAME))
        print('[+] Model trained and saved at ' + SAVE_MODEL_FILENAME)

    def load_model(self):
        SAVE_MODEL_FILENAME = 'emotion_recognition-21150.data-00000-of-00001'
        if isfile(join(SAVE_DIRECTORY, SAVE_MODEL_FILENAME)):
            #self.model.load(join(SAVE_DIRECTORY, SAVE_MODEL_FILENAME))
            self.model.load(join(SAVE_DIRECTORY,'emotion_recognition-21150'))
            print('[+] Model loaded from ' + SAVE_MODEL_FILENAME)
        else:
            print('[+]nont', join(SAVE_DIRECTORY, SAVE_MODEL_FILENAME))

def show_usage():
    # I din't want to have more dependecies
    print('[!] Usage: python emotion_recognition.py')
    print('\t emotion_recognition.py train \t Trains and saves model with saved dataset')
    print('\t emotion_recognition.py poc \t Launch the proof of concept')

mode = 'train'

if __name__ == "__main__":
#     if len(sys.argv) <= 1:
#         show_usage()
#         exit()
    network = EmotionRecognition()
    if mode == 'train':
        
        network.start_training()
#         network.save_model()
    elif mode == 'poc':
        import poc
    else:
        show_usage()





/home/k123/env/python2.7.12/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[+] Dataset found and loaded
[+] Building CNN
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
face_input (InputLayer)      (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       1280      
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 12, 256)       1024      
_________________________________________________________________
activation_2 (Activation)    (

10092/10092 [==============================] - 7s 726us/step - loss: 1.3730 - acc: 0.8316 - val_loss: 4.2072 - val_acc: 0.4733
Epoch 41/100
10092/10092 [==============================] - 7s 726us/step - loss: 1.3179 - acc: 0.8493 - val_loss: 4.0835 - val_acc: 0.4314
Epoch 42/100
10092/10092 [==============================] - 7s 727us/step - loss: 1.3588 - acc: 0.8461 - val_loss: 3.5984 - val_acc: 0.5062
Epoch 43/100
10092/10092 [==============================] - 7s 726us/step - loss: 1.3003 - acc: 0.8579 - val_loss: 3.3094 - val_acc: 0.5071
Epoch 44/100
10092/10092 [==============================] - 7s 726us/step - loss: 1.2952 - acc: 0.8580 - val_loss: 3.3265 - val_acc: 0.5232
Epoch 45/100
10092/10092 [==============================] - 7s 726us/step - loss: 1.2863 - acc: 0.8588 - val_loss: 3.1690 - val_acc: 0.5000
Epoch 46/100
10092/10092 [==============================] - 7s 726us/step - loss: 1.2260 - acc: 0.8724 - val_loss: 3.5877 - val_acc: 0.4938
Epoch 47/100
10092/10092 [=======

10092/10092 [==============================] - 7s 724us/step - loss: 0.8388 - acc: 0.9324 - val_loss: 4.4159 - val_acc: 0.5018
Epoch 99/100
10092/10092 [==============================] - 7s 724us/step - loss: 0.8406 - acc: 0.9354 - val_loss: 4.1839 - val_acc: 0.4661
Epoch 100/100
10092/10092 [==============================] - 7s 724us/step - loss: 0.7814 - acc: 0.9442 - val_loss: 3.9209 - val_acc: 0.5401
